In [1]:
import pandas as pd 
import numpy as np 
import re 
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk 
import warnings  
warnings.filterwarnings("ignore", category=DeprecationWarning)
import keras
from keras.models import Sequential 
from keras.layers import Embedding,Flatten,Dense,SpatialDropout1D,LSTM
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [2]:
import os 
print(os.listdir(r"C:\Users\Ali\Desktop\data sentiment")) 

['4_5796360327922715229.pdf', 'data.txt', 'first', 'Git.pdf', 'Managing_environments_—_conda_4_10_1_post10+f47dc5e6f_documentation.pdf', 'minst code', 'NLP - Twitter Sentiment Analysis Project _ Kaggle.pdf', 'PEP 8 -- Style Guide for Python Code _ Python.org.pdf', 'project four', 'sesoin four']


In [3]:
train_df = pd.read_csv(r"C:\Users\Ali\Desktop\data sentiment\project four\New folder\c.csv") 
test_df = pd.read_csv(r"C:\Users\Ali\Desktop\data sentiment\project four\New folder\c.csv") 

In [4]:
train_df.head(10)

,apple,positive,126415614616154112
0,apple,positive,126404574230740992
1,apple,positive,126402758403305474
2,apple,positive,126397179614068736
3,apple,positive,126395626979196928
4,apple,positive,126394830791254016
5,apple,positive,126379685453119488
6,apple,positive,126377656416612353
7,apple,positive,126373779483004928
8,apple,positive,126366353757179904
9,apple,positive,126366123368267776


In [5]:
test_df.head(10)

,apple,positive,126415614616154112
0,apple,positive,126404574230740992
1,apple,positive,126402758403305474
2,apple,positive,126397179614068736
3,apple,positive,126395626979196928
4,apple,positive,126394830791254016
5,apple,positive,126379685453119488
6,apple,positive,126377656416612353
7,apple,positive,126373779483004928
8,apple,positive,126366353757179904
9,apple,positive,126366123368267776


In [6]:
"clean the data"
"A converting html entites"

from html.parser import HTMLParser 
html_parser = HTMLParser() 
 


In [7]:
test_df['clean_tweet'] = test_df["TweetText"].apply(lambda x: html_parser.unescape(x)) 
test_df.head(10) 

KeyError: 'TweetText'

In [ ]:
 "B Removing @user from all the tweets"
def remove_pattern(input_txt, pattern):    
    r = re.findall(pattern, input_txt)    
    for i in r:        
        input_txt = re.sub(i, '', input_txt)    
    return input_txt 
    


In [ ]:
test_df["clean_tweet"]=np.vectorize(remove_pattern)(test_df["clean_tweet"],"@[\w]*")
test_df.head(10)

In [ ]:
"C Changing all the tweets into lowercase"
test_df["clean_tweet"]=test_df["clean_tweet"].apply(lambda x:x.lower())
test_df.head(10)

In [ ]:
"D Apostrophe Lookup"
apostrophe_dict = { "ain't": "am not / are not",
                   "aren't": "are not / am not", 
                   "can't": "cannot", 
                   "can't've": "cannot have", 
                   "'cause": "because", 
                   "could've": "could have", 
                   "couldn't": "could not", 
                   "couldn't've": "could not have", 
                   "didn't": "did not", 
                   "doesn't": "does not", 
                   "don't": "do not", 
                   "hadn't": "had not", 
                   "hadn't've": "had not have", 
                   "hasn't": "has not", 
                   "haven't": "have not", 
                   "he'd": "he had / he would", 
                   "he'd've": "he would have", 
                   "he'll": "he shall / he will", 
                   "he'll've": "he shall have / he will have", 
                   "he's": "he has / he is", 
                   "how'd": "how did", 
                   "how'd'y": "how do you", 
                   "how'll": "how will", 
                   "how's": "how has / how is", 
                   "i'd": "I had / I would", 
                   "i'd've": "I would have", 
                   "i'll": "I shall / I will", 
                   "i'll've": "I shall have / I will have", 
                   "i'm": "I am", 
                   "i've": "I have", 
                   "isn't": "is not", 
                   "it'd": "it had / it would", 
                   "it'd've": "it would have", 
                   "it'll": "it shall / it will", 
                   "it'll've": "it shall have / it will have", 
                   "it's": "it has / it is", 
                   "let's": "let us", 
                   "ma'am": "madam", 
                   "mayn't": "may not",
                   "might've": "might have", 
                   "mightn't": "might not", 
                   "mightn't've": "might not have", 
                   "must've": "must have", 
                   "mustn't": "must not", 
                   "mustn't've": "must not have", 
                   "needn't": "need not", 
                   "needn't've": "need not have", 
                   "o'clock": "of the clock", 
                   "oughtn't": "ought not", 
                   "oughtn't've": "ought not have", 
                   "shan't": "shall not", 
                   "sha'n't": "shall not", 
                   "shan't've": "shall not have", 
                   "she'd": "she had / she would", 
                   "she'd've": "she would have", 
                   "she'll": "she shall / she will", 
                   "she'll've": "she shall have / she will have", 
                   "she's": "she has / she is", 
                   "should've": "should have", 
                   "shouldn't": "should not", 
                   "shouldn't've": "should not have", 
                   "so've": "so have", 
                   "so's": "so as / so is",
                   "that'd": "that would / that had", 
                   "that'd've": "that would have", 
                   "that's": "that has / that is", 
                   "there'd": "there had / there would", 
                   "there'd've": "there would have",
                   "there's": "there has / there is",
                   "they'd": "they had / they would", 
                   "they'd've": "they would have", 
                   "they'll": "they shall / they will", 
                   "they'll've": "they shall have / they will have", 
                   "they're": "they are", 
                   "they've": "they have", 
                   "to've": "to have", 
                   "wasn't": "was not", 
                   "we'd": "we had / we would", 
                   "we'd've": "we would have",
                   "we'll": "we will", 
                   "we'll've": "we will have", 
                   "we're": "we are", 
                   "we've": "we have", 
                   "weren't": "were not", 
                   "what'll": "what shall / what will", 
                   "what'll've": "what shall have / what will have",
                   "what're": "what are", 
                   "what's": "what has / what is", 
                   "what've": "what have", 
                   "when's": "when has / when is", 
                   "when've": "when have", 
                   "where'd": "where did", 
                   "where's": "where has / where is", 
                   "where've": "where have", 
                   "who'll": "who shall / who will",
                   "who'll've": "who shall have / who will have", 
                   "who's": "who has / who is", 
                   "who've": "who have", 
                   "why's": "why has / why is", 
                   "why've": "why have", 
                   "will've": "will have", 
                   "won't": "will not", 
                   "won't've": "will not have", 
                   "would've": "would have", 
                   "wouldn't": "would not", 
                   "wouldn't've": "would not have", 
                   "y'all": "you all", 
                   "y'all'd": "you all would", 
                   "y'all'd've": "you all would have", 
                   "y'all're": "you all are", 
                   "y'all've": "you all have", 
                   "you'd": "you had / you would", 
                   "you'd've": "you would have", 
                   "you'll": "you shall / you will", 
                   "you'll've": "you shall have / you will have", 
                   "you're": "you are", 
                   "you've": "you have" 
                  } 
apostrophe_dict


In [ ]:
def lookup_dict(text, dictionary):    
    for word in text.split():        
        if word.lower() in dictionary:            
            if word.lower() in text.split():               
                text = text.replace(word, dictionary[word.lower()])    
    return text 

In [ ]:
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: lookup_dict(x,apostrophe_dict)) 
test_df.head(10) 

In [ ]:
short_word_dict = {
    "121": "one to one", 
    "a/s/l": "age, sex, location", 
    "adn": "any day now", 
    "afaik": "as far as I know", 
    "afk": "away from keyboard", 
    "aight": "alright", 
    "alol": "actually laughing out loud", 
    "b4": "before", 
    "b4n": "bye for now", 
    "bak": "back at the keyboard", 
    "bf": "boyfriend", 
    "bff": "best friends forever", 
    "bfn": "bye for now", 
    "bg": "big grin", 
    "bta": "but then again", 
    "btw": "by the way", 
    "cid": "crying in disgrace", 
    "cnp": "continued in my next post", 
    "cp": "chat post", 
    "cu": "see you", 
    "cul": "see you later", 
    "cul8r": "see you later", 
    "cya": "bye", 
    "cyo": "see you online", 
    "dbau": "doing business as usual", 
    "fud": "fear, uncertainty, and doubt", 
    "fwiw": "for what it's worth", 
    "fyi": "for your information", 
    "g": "grin", 
    "g2g": "got to go", 
    "ga": "go ahead", 
    "gal": "get a life", 
    "gf": "girlfriend", 
    "gfn": "gone for now", 
    "gmbo": "giggling my butt off", 
    "gmta": "great minds think alike", 
    "h8": "hate", "hagn": "have a good night", 
    "hdop": "help delete online predators", 
    "hhis": "hanging head in shame",
    "iac": "in any case", 
    "ianal": "I am not a lawyer", 
    "ic": "I see", 
    "idk": "I don't know", 
    "imao": "in my arrogant opinion", 
    "imnsho": "in my not so humble opinion",
    "imo": "in my opinion", 
    "iow": "in other words", 
    "ipn": "I’m posting naked",
    "irl": "in real life", 
    "jk": "just kidding",
    "l8r": "later",
    "ld": "later, dude", 
    "ldr": "long distance relationship", 
    "llta": "lots and lots of thunderous applause",
    "lmao": "laugh my ass off", 
    "lmirl": "let's meet in real life", 
    "lol": "laugh out loud",
    "ltr": "longterm relationship",
    "lulab": "love you like a brother",
    "lulas": "love you like a sister", 
    "luv": "love", 
    "m/f": "male or female", 
    "m8": "mate", 
    "milf": "mother I would like to fuck", 
    "oll": "online love", 
    "omg": "oh my god", 
    "otoh": "on the other hand", 
    "pir": "parent in room", 
    "ppl": "people", 
    "r": "are",
    "rofl": "roll on the floor laughing", 
    "rpg": "role playing games", 
    "ru": "are you", 
    "shid": "slaps head in disgust",
    "somy": "sick of me yet",
    "sot": "short of time", 
    "thanx": "thanks", 
    "thx": "thanks", 
    "ttyl": "talk to you later",
    "u": "you", 
    "ur": "you are",
    "uw": "you’re welcome", 
    "wb": "welcome back", 
    "wfm": "works for me", 
    "wibni": "wouldn't it be nice if", 
    "wtf": "what the fuck", 
    "wtg": "way to go",
    "wtgp": "want to go private", 
    "ym": "young man", 
    "gr8": "great" 
}   

In [ ]:
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: lookup_dict(x,short_word_dict)) 
test_df.head(10) 


In [ ]:
" F  Emoticon Lookup"
emoticon_dict = { ":)": "happy", 
                 ":‑)": "happy", 
                 ":-]": "happy", 
                 ":-3": "happy", 
                 ":->": "happy", 
                 "8-)": "happy", 
                 ":-}": "happy", 
                 ":o)": "happy", 
                 ":c)": "happy", 
                 ":^)": "happy", 
                 "=]": "happy", 
                 "=)": "happy", 
                 "<3": "happy", 
                 ":-(": "sad", 
                 ":(": "sad", 
                 ":c": "sad", 
                 ":<": "sad",
                 ":[": "sad", 
                 ">:[": "sad", 
                 ":{": "sad", 
                 ">:(": "sad",
                 ":-c": "sad", 
                 ":-< ": "sad",
                 ":-[": "sad", 
                 ":-||": "sad" 
                } 


In [ ]:
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: lookup_dict(x,emoticon_dict)) 
test_df.head(10) 



In [ ]:
"G ReplacingPunctuations with space"
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: re.sub(r'[^\w\s]',' ',x)) 
test_df.head(10) 


In [ ]:
import re
s = "stringeeee322#@!! With. Punctuation?"
s = re.sub(r'[^\w\s]','',s)
print (s)

In [ ]:
"Replacing Special Characters with space"
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
test_df.head(10) 



In [ ]:
"J  Removing words whom length is 1"
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
test_df['clean_tweet'][0:5] 



In [ ]:
"K  Spelling Correction - With TextBlob Library"
from textblob import TextBlob
text = test_df['clean_tweet'][0:10].apply(lambda x: str(TextBlob(x).correct())) 
print(text)


In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [ ]:
test_df['tweet_token'] = test_df['clean_tweet'].apply(lambda x: word_tokenize(x))
test_df.head(10)

In [ ]:
stop_words = set(stopwords.words('english')) 
stop_words ={
    'a',
    'about',
    'above', 
    'after',
    'again',
    'against',
    'ain',
    'all',
    'am',
    'an',
    'and',
    'any', 
    'are',
    'aren',
    "aren't",
    'as',
    'at',
    'be',
    'because',
    'been',
    'before', 
    'being',
    'below',  
    'between',
    'both',
    'but',
    'by', 
    'can', 
    'couldn',
    "couldn't",
    'd',
    'did',
    'didn',
    "didn't",
    'do',
    'does',  
    'doesn',
    "doesn't",
    'doing',
    'don',
    "don't", 
    'down',
    'during',  
    'each', 
    'few',
    'for',
    'from', 
    'further',
    'had',
    'hadn',
    "hadn't",
    'has',
    'hasn',  
    "hasn't",
    'have',
    'haven',
    "haven't",
    'having',
    'he', 
    'her',  
    'here',
    'hers',
    'herself',
    'him', 
    'himself',
    'his', 
    'how',
    'i',
    'if',
    'in',  
    'into',
    'is', 
    'isn',
    "isn't", 
    'it', 
    "it's",  
    'its',
    'itself',
    'just',
    'll',
    'm',
    'ma',
    'me',
    'mightn', 
    "mightn't", 
    'more',
    'most',
    'mustn', 
    "mustn't",  
    'my', 
    'myself',
    'needn',  
    "needn't", 
    'no',
    'nor', 
    'not',  
    'now',
    'o',
    'of',
    'off', 
    'on',  
    'once',
    'only',
    'or',
    'other',
    'our', 
    'ours',
    'ourselves',
    'out',
    'over',  
    'own', 
    're',
    's',
    'same',
    'shan',  
    "shan't",
    'she', 
    "she's",
    'should', 
    "should've",
    'shouldn', 
    "shouldn't",
    'so', 
    'some',
    'such',
     't', 
    'than', 
    'that',
    "that'll",
    'the', 
    'their',  
    'theirs', 
    'them',  
    'themselves',
    'then',
    'there',  
    'these', 
    'they', 
    'this',
    'those',  
    'through',
    'to',
    'too', 
    'under', 
    'until',
    'up',
    've',
    'very',  
    'was',
    'wasn', 
    "wasn't",
    'we',
    'were',  
    'weren',
    "weren't",
    'what', 
    'when', 
    'where',  
    'which',
    'while',  
    'who',
    'whom',
    'why',
    'will',
    'with', 
    'won', 
    "won't",
     't', 
    'than',
    'that',
    "that'll",
    'the', 
    'their',  
    'theirs', 
    'them',
    'themselves',
    'then',  
    'there', 
    'these',  
    'they',
    'this',  
    'those',
    'through',
    'to',
    'too', 
    'under',
    'until',  
    'up',  
    've',  
    'very',  
    'was',  
    'wasn',
    "wasn't",
    'we', 
    'were',
    'weren', 
    "weren't", 
    'what', 
    'when',  
    'where', 
    'which',
    'while', 
    'who', 
    'whom',  
    'why',
    'will', 
    'with',
    'won',
    "won't",
}

In [ ]:
test_df['tweet_token_filtered'] = test_df['tweet_token'].apply(lambda x: [word for word in x if not word in stop_words]) 

In [ ]:
test_df[['tweet_token', 'tweet_token_filtered']].head(10) 

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
stemming=PorterStemmer()

In [ ]:
test_df['tweet_stemmed'] = test_df['tweet_token_filtered'].apply(lambda x: ' '.join([stemming.stem(i) for i in x])) 
test_df['tweet_stemmed'].head(10) 


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer 
lemmatizing = WordNetLemmatizer() 

In [ ]:
test_df['tweet_lemmatized'] = test_df['tweet_token_filtered'].apply(lambda x:' '.join([lemmatizing.lemmatize(i) for i in x])) 
test_df['tweet_lemmatized'].head(10) 

In [ ]:
test_df.head(10)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
cv= CountVectorizer() 
print(cv)

In [ ]:
r=cv.fit(test_df["tweet_lemmatized"])
print(r)

In [ ]:
x=cv.transform(test_df["tweet_lemmatized"])
y=test_df["Sentiment"]
print(x)

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [ ]:
x_trian,x_test,y_train,y_test = train_test_split(x,y,train_size=0.80)
print(x_trian.shape)
print(y_train.shape)

In [ ]:
import keras
model=keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256,activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dense(2,activation="softmax"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(x_trian,y_trian,epochs=3)

In [ ]:
""""for c in [0.1,0.05,0.25,0.32,0.5,3]:
    lr=LogisticRegression(C=c)
    lr.fit(x_trian,y_train)
    print('accuracy for c=%s:%s'%(c,accuracy_score(y_test,lr.predict(x_test))))"""

In [ ]:
""""for c in [0.1,0.05,0.25,0.32,0.5,0.0000002]:
    sv=SVC(C=c,kernel='poly',degree=3,gamma=10)
    sv.fit(x_trian,y_train)
    print('accuracy for c=%s:%s'%(c,accuracy_score(y_test,lr.predict(x_test))))"""